In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json

drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json


In [15]:
!ls /content/drive/MyDrive/ndui

best_model_albuquerque.pth  best_model_shanghai.pth	  dmsp_1992_2022_tokyo.nc
best_model_berlin.pth	    best_model_tokyo.pth	  dmsp_2012_austin.nc
best_model_cairo.pth	    best_model_washington_dc.pth  dmsp_swinIR_1992_2022_delhi.nc
best_model_columbus.pth     cur_model_epoch0.h5		  dmsp_swinIR_1992_2022_dubai.nc
best_model_delhi.pth	    cur_model_epoch1.h5		  dmsp_swinIR_1992_2022_hongkong.nc
best_model_denver.pth	    cur_model_epoch2.h5		  dmsp_swinIR_1992_2022_london.nc
best_model_dhaka.pth	    cur_model_epoch3.h5		  dmsp_swinIR_1992_2022_melbourne.nc
best_model_dubai.pth	    cur_model_epoch4.h5		  Guangzhou_studyarea.shp
best_model_hongkong.pth     cur_model_epoch5.h5		  Guangzhou_studyarea.shx
best_model_kansas_city.pth  cur_model_epoch6.h5		  model_MAE_depth_4_per_level_3.h5
best_model_la.pth	    cur_model_epoch7.h5		  ndui_new_york.nc
best_model_london.pth	    cur_model_epoch8.h5		  referenceData.cpg
best_model_louisville.pth   cur_model_epoch9.h5		  referenceData.dbf
best_model

In [3]:
import ee
import os

# Set the path to the service account key file
service_account = 'editor@ee-manmeet20singh15-wbis.iam.gserviceaccount.com'
key_file = 'drive/MyDrive/earth_engine/ee-manmeet20singh15-wbis-fab7f1ca35e0.json'

# Use the service account for authentication
credentials = ee.ServiceAccountCredentials(service_account, key_file)
ee.Initialize(credentials)

In [4]:
!pip install timm
!pip install wxee

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.9 MB/s eta 0:00:00


In [5]:
import ee
import wxee
# wxee.Initialize()
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Resize
from PIL import Image
import os
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.checkpoint as checkpoint
from timm.models.layers import DropPath, to_2tuple, trunc_normal_
import numpy as np
import xarray as xr

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Resize
from PIL import Image
import os
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [6]:
import pandas as pd

# Data provided
cities = ['delhi', 'hongkong', 'melbourne', 'dubai', 'london', 'berlin', 'tokyo', 'dhaka', 'shanghai', 'saopaulo', 'mexicocity', 'cairo']
lats_  = [28.7041, 22.3193, -37.8136, 25.2048, 51.5072, 52.5200, 35.6764, 23.8041, 31.2304, -23.5558, 19.4326, 30.0444]
lons_  = [77.1025, 114.1694, 144.9631, 55.2708, -0.1276, 13.4050, 139.6500, 90.4152, 121.4737, -46.6396, -99.1332, 31.2357]

cities = ['delhi', 'hongkong', 'melbourne', 'dubai', 'london']
lats_  = [28.7041, 22.3193, -37.8136, 25.2048, 51.5072]
lons_  = [77.1025, 114.1694, 144.9631, 55.2708, -0.1276]

# Creating DataFrame
df = pd.DataFrame({'Name': cities, 'lat': lats_, 'lon': lons_})
df

,Name,lat,lon
0,delhi,28.7041,77.1025
1,hongkong,22.3193,114.1694
2,melbourne,-37.8136,144.9631
3,dubai,25.2048,55.2708
4,london,51.5072,-0.1276


In [29]:
aoi_city = []
for i in range(len(df.lat)):
  lats, late = df.lat[i]-0.25, df.lat[i]+0.25
  lons, lone = df.lon[i]-0.25, df.lon[i]+0.25
  aoi = ee.Geometry.Polygon(
        [[[lons, lats],
          [lone, lats],
          [lone, late],
          [lons, late]]])
  aoi_city.append(aoi)
aoi_city[0]

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              76.8525,
              28.4541
            ],
            [
              77.3525,
              28.4541
            ],
            [
              77.3525,
              28.9541
            ],
            [
              76.8525,
              28.9541
            ]
          ]
        ]
      },
      "evenOdd": {
        "constantValue": true
      }
    }
  }
})

In [30]:
coords = aoi_city[0].coordinates().getInfo()[0]
coords

[[76.8525, 28.4541],
 [77.3525, 28.4541],
 [77.3525, 28.9541],
 [76.8525, 28.9541],
 [76.8525, 28.4541]]

In [ ]:
# ndui_1999_2022 for Listed Cities
dates = pd.date_range('1999', '2022', freq='YS')
ndui_city = []
for i in range(len(df.lat)):
    print(cities[i])
    ndui_c_ = []
    for year_ in range(1999,2023):
        print(year_)
        year = str(year_)#'1999'
        # ds_dmsp_interp = ds_dmsp.sel(time=slice(year, year)).interp(lon=ds_ndvi.x, lat=ds_ndvi.y).dmsp.values[0,:,:]/63.0
        # L7 = ee.ImageCollection('LE7_L1T_TOA').filterDate(year+'-01-01', year+'-12-31')
        if year_ == 1999:
            L7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").filterDate(str(year_)+'-01-01', str(year_+2)+'-12-31')
            print(year_)
            print(str(year_)+'-01-01', str(year_+2)+'-12-31')
        elif year_ == 2000:
            L7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").filterDate(str(year_-1)+'-01-01', str(year_+1)+'-12-31')
            print(year_)
            print(str(year_-1)+'-01-01', str(year_+1)+'-12-31')
        else:
            L7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_TOA").filterDate(str(year_-2)+'-01-01', str(year_)+'-12-31')
            print(year_)
            print(str(year_-2)+'-01-01', str(year_)+'-12-31')

        def fun4(img):
            bad1 = img.select('B1').eq(0.0)
            bad2 = img.select('B2').eq(0.0)
            bad3 = img.select('B3').eq(0.0)
            bad4 = img.select('B4').eq(0.0)
            bad5 = img.select('B5').eq(0.0)
            bad7 = img.select('B7').eq(0.0)
            mask = img.mask().And(bad1.Or(bad2).Or(bad3).Or(bad4).Or(bad5).Or(bad7).Not())
            #var mask = img.select('10','20','30','40','50','70').mask().reduce('product').eq(1);
            masked = img.mask(mask);
            ndvi = masked.normalizedDifference(["B4","B3"])
            return ndvi

        NDVIs = L7.map(fun4)

        Mean_NDVI = NDVIs.median()
        Max_NDVI = NDVIs.max()
        Min_NDVI = NDVIs.min()
        mosaic = Mean_NDVI.where(Max_NDVI.gt(0.4), Max_NDVI)
        mosaic = mosaic.where(Min_NDVI.lt(-0.2), Min_NDVI)

        mosaic = mosaic.set('system:time_start', 0)
        ds_ndvi_c = mosaic.wx.to_xarray(region=aoi_city[i].bounds(), scale=30)
        ds_ndvi_c_ = ds_ndvi_c.nd.values[0,:,:]
        ds_dmsp_1992_2022_city = xr.open_dataset('/content/drive/MyDrive/ndui/dmsp_swinIR_1992_2022_'+cities[i]+'.nc')
        ds_dmsp_interp_c = ds_dmsp_1992_2022_city.sel(time=slice(year, year)).interp(lon=ds_ndvi_c.x, lat=ds_ndvi_c.y).dmsp.values[0,:,:]/63.0
        # ds_ndvi_c_ = mosaic.wx.to_xarray(region=aoi_city[i].bounds(), scale=30).nd.values[0,:,:]

        ndui_c = (ds_dmsp_interp_c - ds_ndvi_c_)/(ds_dmsp_interp_c + ds_ndvi_c_)
        ndui_c[ndui_c>1.0] = 1.0
        ndui_c[ndui_c<-1.0] = -1.0
        ndui_c_.append(ndui_c)
    print(len(ndui_c_))
    print(np.stack(ndui_c_).shape)

    ndui_c_stack = np.stack(ndui_c_)
    ds_ndui_c = xr.Dataset({
        'ndui': xr.DataArray(
            data   = ndui_c_stack,   # enter data here
            dims   = ['time', 'lat', 'lon'],
            coords = {'time':dates, 'lat': ds_ndvi_c.y.values, 'lon': ds_ndvi_c.x.values},
            ),
        },
                                               )
    print(ds_ndui_c)
    ndui_city.append(ds_ndui_c)
ndui_city[0]

delhi
1999
1999
1999-01-01 2001-12-31


Downloading:   0%|          | 0.00/8.51M [00:00<?, ?iB/s]

2000
2000
1999-01-01 2001-12-31


Downloading:   0%|          | 0.00/8.51M [00:00<?, ?iB/s]

2001
2001
1999-01-01 2001-12-31


Downloading:   0%|          | 0.00/8.51M [00:00<?, ?iB/s]

2002
2002
2000-01-01 2002-12-31


Downloading:   0%|          | 0.00/8.70M [00:00<?, ?iB/s]

2003
2003
2001-01-01 2003-12-31


In [ ]:
for i in range(len(df.lat)):
  ndui_city[i].to_netcdf('ndui_1992_2022_'+cities[i]+'.nc')

In [ ]:
!mv ndui_1992_2022_*.nc /content/drive/MyDrive/ndui